In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score, classification_report,auc
%matplotlib inline
import keras
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Bidirectional,Dense, LSTM, Activation, Dropout, Flatten
from keras.layers import LeakyReLU
from sklearn.metrics import accuracy_score
import tensorflow as tf
from keras.regularizers import l2
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
import tensorflow as tf
import tensorflow as tf
import keras
from keras.regularizers import l2
from keras.models import Model
from keras.applications import mobilenet_v2, mobilenet, resnet50, densenet
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, \
    BatchNormalization, Activation, GlobalAveragePooling2D, DepthwiseConv2D, \
    Dropout, ReLU, Concatenate, Input, add

from keras.callbacks import ModelCheckpoint
from keras import backend as K

import matplotlib.pyplot as plt
import math
from keras.callbacks import CSVLogger
from datetime import datetime

import cv2
import os.path
from keras.utils.vis_utils import plot_model
from scipy.spatial import distance
import scipy.io as sio
import tensorflow as tf

df_test = pd.read_csv("All_feature_Independent_dataframe.csv")

X_test_NetSurfP2 = df_test.iloc[:,2100:2428]

y_test_NetSurfP2 = df_test["Target"]


print(X_test_NetSurfP2.isnull().sum().sum())
X_test_NetSurfP2.fillna(0,inplace=True)
print(X_test_NetSurfP2.isnull().sum().sum())

scaler = MinMaxScaler()


X_test_NetSurfP2 = scaler.fit_transform(X_test_NetSurfP2)


X_test_NetSurfP2 = np.array(X_test_NetSurfP2)
y_test_NetSurfP2 = np.array(y_test_NetSurfP2)

X_test_PSSM_test = df_test.iloc[:,1280:2100]

y_test_PSSM = df_test["Target"]

print(X_test_PSSM_test.isnull().sum().sum())
X_test_PSSM_test.fillna(0,inplace=True)
print(X_test_PSSM_test.isnull().sum().sum())

scaler = MinMaxScaler()


X_test_PSSM_test = scaler.fit_transform(X_test_PSSM_test)


X_test_PSSM_test = np.array(X_test_PSSM_test)
y_test_PSSM = np.array(y_test_PSSM)

X_test_Gapped_Dipeptide = df_test.iloc[:,2428:2468]

y_test_Gapped_Dipeptide = df_test["Target"]


scaler = MinMaxScaler()


X_test_Gapped_Dipeptide = scaler.fit_transform(X_test_Gapped_Dipeptide)


X_test_Gapped_Dipeptide = np.array(X_test_Gapped_Dipeptide)
y_test_Gapped_Dipeptide = np.array(y_test_Gapped_Dipeptide)

X_test_PSSM_NetSurfP2_Gapped_Dipeptide = np.hstack((X_test_PSSM_test,X_test_NetSurfP2,X_test_Gapped_Dipeptide))

y_test_PSSM_NetSurfP2_Gapped_Dipeptide = y_test_Gapped_Dipeptide

X_test = X_test_PSSM_NetSurfP2_Gapped_Dipeptide

y_test = y_test_PSSM_NetSurfP2_Gapped_Dipeptide




test_pad = np.zeros((447,108))
X_test = np.hstack([X_test,test_pad])

X_test = X_test.reshape(-1,36,36)

print(X_test.shape,y_test.shape)



test = []
for i in range(X_test.shape[0]):
    arg = tf.convert_to_tensor(X_test[i], dtype=tf.float32)
    arg = tf.expand_dims(arg,-1)
#     value = tf.image.grayscale_to_rgb(arg)
    test.append(arg.numpy()) 

test = np.array(test)

model = keras.models.load_model("Unet_20_glycosyaltion5__.h5")

Y_pred = model.predict(test)
Y_pred = (Y_pred > 0.5)
y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
y_pred = np.array(y_pred)
print("Matthews Correlation : ",matthews_corrcoef(y_test, y_pred))
print("Confusion Matrix : \n",confusion_matrix(y_test, y_pred))
print("Accuracy on test set:   ",accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)

TP = cm[1][1]
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]

mcc = matthews_corrcoef(y_test, y_pred)

Sensitivity = TP/(TP+FN)

Specificity = TN/(TN+FP)

print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)

print(classification_report(y_test, y_pred))

1680
0
4200
0
(447, 36, 36) (447,)
Matthews Correlation :  0.5191837900925341
Confusion Matrix : 
 [[185  95]
 [ 21 146]]
Accuracy on test set:    0.7404921700223713
Sensitivity:    0.874251497005988 	 Specificity:    0.6607142857142857
              precision    recall  f1-score   support

           0       0.90      0.66      0.76       280
           1       0.61      0.87      0.72       167

    accuracy                           0.74       447
   macro avg       0.75      0.77      0.74       447
weighted avg       0.79      0.74      0.74       447

